In [ ]:
import gym
import numpy as np
import torch
import time
from collections  import deque
from parallelEnv import parallelEnv
from envs import make_vec_envs
from model import Policy
from utils import get_render_func, get_vec_normalize

device = torch.device("cpu")
print('device: ', device)
model_path = "model_zoo/common_model_270"
seed = 0 

## model Policy uses MLPBase
envs = parallelEnv('MountainCarContinuous-v0', n=8, seed=seed) ## weights created by n = 16

max_steps = envs.max_steps
print('max_steps: ', max_steps)

policy = Policy(envs.observation_space.shape, envs.action_space,\
        base_kwargs={'recurrent': False})

print('policy: ', policy)
policy.to(device)

num_processes = 1
env_venv = make_vec_envs('MountainCarContinuous-v0', \
                    seed + 1000, num_processes,
                    None, None, False, device=device, allow_early_resets=False)

print('envs.observation_space.shape: ', envs.observation_space.shape, \
      ', len(obs_shape): ', len(envs.observation_space.shape))
print('envs.action_space: ',  envs.action_space, \
      ', action_space.shape[0]: ', envs.action_space.shape[0])


def load_test(model):
    model.base = torch.load(model_path + '/we0_model_pre_train.pth')
    model.base.actor.load_state_dict(torch.load(model_path + '/we0_actor_final.pth'))
    # model.base.critic.load_state_dict(torch.load('dir_save\we0_critic_final.pth'))
    # model.base.critic_linear.load_state_dict(torch.load('dir_save\we0_critic_linear_final.pth'))
    model.dist = torch.load(model_path + '/we0_model_dist_final.pth')
    
load_test(model = policy)    


In [ ]:
## No CUDA, only CPU

def play_VecEnv(env, model, num_episodes):

    obs = env.reset()
    obs = torch.Tensor(obs)
    obs = obs.float()
        
    recurrent_hidden_states = torch.zeros(1, model.recurrent_hidden_state_size)
    
    masks = torch.zeros(1, 1)
    
    scores_deque = deque(maxlen=100)

    render_func = get_render_func(env)
        
    for i_episode in range(1, num_episodes+1):     

        time_start = time.time()
        total_reward = np.zeros(num_processes)
        timestep = 0

        done = False
        
        while not done:
        
            with torch.no_grad():
                value, action, _, recurrent_hidden_states = \
                    model.act(obs, recurrent_hidden_states, masks, deterministic=False) # obs = state
                            

            render_func()
            
            obs, reward, done, _ = env.step(action.unsqueeze(1))
            obs = torch.Tensor(obs)
            obs = obs.float()

            reward = reward.detach().numpy()
            masks.fill_(0.0 if done else 1.0)
            
            total_reward += np.mean(reward)
            
            time.sleep(0.04)
            
            timestep += 1
            
            if done.all() == True or timestep + 1 == max_steps: ##   999:
                break

        s = (int)(time.time() - time_start)
        
        scores_deque.append(total_reward)        
        avg_score = np.mean(scores_deque)
                    
        print('Episode {} \tScore: {:.2f}, Avg.Score: {:.2f}, \tTime: {:02}:{:02}:{:02}'\
                  .format(i_episode, np.mean(total_reward), avg_score,  s//3600, s%3600//60, s%60))
    
play_VecEnv(env=env_venv, model=policy, num_episodes=7)   



In [6]:
env_venv.close()